# Importar las librerias

In [1]:
from datetime import datetime
import MetaTrader5 as mt5
import pandas as pd
import pytz
import talib as ta
from backtesting import Strategy, Backtest

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)

# Obtener los datos

In [2]:

# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)
 
# establish connection to MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
timezone = pytz.timezone("Etc/UTC")
utc_from = datetime(2014, 3, 1, tzinfo=timezone)
utc_to = datetime(2024, 9, 1, tzinfo=timezone)
rates = mt5.copy_rates_range("EURUSDm", mt5.TIMEFRAME_D1, utc_from, utc_to)
 
mt5.shutdown()
 
df = pd.DataFrame(rates)

df['time'] = pd.to_datetime(df['time'], unit='s')
df = df.rename(columns={
    'time':'Date',
    'open':'Open',
    'high':'High',
    'low':'Low',
    'close':'Close',
})

df = df.set_index('Date')
df

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288


,Open,High,Low,Close,tick_volume,spread,real_volume
Date,,,,,,,
2014-03-02,1.37803,1.37870,1.37529,1.37732,4650,0,0
2014-03-03,1.37736,1.37922,1.37254,1.37345,60747,0,0
2014-03-04,1.37346,1.37808,1.37200,1.37410,58371,0,0
2014-03-05,1.37402,1.37479,1.37068,1.37307,50682,0,0
2014-03-06,1.37304,1.38724,1.37208,1.38624,71416,0,0
...,...,...,...,...,...,...,...
2024-08-27,1.11641,1.11900,1.11494,1.11766,24552,8,0
2024-08-28,1.11767,1.11797,1.11046,1.11241,28228,8,0
2024-08-29,1.11239,1.11394,1.10553,1.10797,35597,7,0


In [3]:
data_percent = int(df.shape[0] * 0.2)
data_percent

647

In [4]:
df_min = df.iloc[:data_percent]
df_min

,Open,High,Low,Close,tick_volume,spread,real_volume
Date,,,,,,,
2014-03-02,1.37803,1.37870,1.37529,1.37732,4650,0,0
2014-03-03,1.37736,1.37922,1.37254,1.37345,60747,0,0
2014-03-04,1.37346,1.37808,1.37200,1.37410,58371,0,0
2014-03-05,1.37402,1.37479,1.37068,1.37307,50682,0,0
2014-03-06,1.37304,1.38724,1.37208,1.38624,71416,0,0
...,...,...,...,...,...,...,...
2016-03-22,1.12342,1.12596,1.11883,1.12205,96595,0,0
2016-03-23,1.12196,1.12196,1.11586,1.11772,123045,0,0
2016-03-24,1.11771,1.11871,1.11434,1.11640,116630,0,0


# Test preliminar

In [12]:
class ShortMomentum(Strategy):
    risk = 1

    rsi_upper_threshold = 90
    rsi_lower_threshold = 10
    
    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, timeperiod=2)
            
    def next(self):
        
        if self.position:

            # Stop and reverse si hay perdidas
            if self.position.is_long and self.trades[-1].pl < 0:
                
                self.position.close()
                self.sell(
                    size=self.risk / 100, 
                )

            if self.position.is_short and self.trades[-1].pl < 0:
                
                self.position.close()
                self.buy(
                    size=self.risk / 100, 
                )
            
            if self.position.is_long and self.rsi[-1] >= self.rsi_upper_threshold:
                self.position.close()
                
            if self.position.is_short and self.rsi[-1] <= self.rsi_lower_threshold:
                self.position.close()
                        
        else:
            
            if self.rsi[-1] < self.rsi_lower_threshold or self.rsi[-1 > self.rsi_upper_threshold]:
                return
            
            
            if self.data.Close[-1] > self.data.Close[-2]:
                self.buy(
                    size=self.risk / 100, 
                )
            
            if self.data.Close[-1] < self.data.Close[-2]:
                self.sell(
                    size=self.risk / 100, 
                )

In [16]:
bt = Backtest(
    df_min, 
    ShortMomentum, 
    cash=100_000, 
    commission=7e-4
)

stats = bt.run()

bt.plot()

stats

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2014-03-02 00:00:00
End                       2016-03-27 00:00:00
Duration                    756 days 00:00:00
Exposure Time [%]                    90.57187
Equity Final [$]                 99876.257944
Equity Peak [$]                 100004.561146
Return [%]                          -0.123742
Buy & Hold Return [%]              -18.997764
Return (Ann.) [%]                   -0.048215
Volatility (Ann.) [%]                0.081509
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.215461
Avg. Drawdown [%]                    -0.04659
Max. Drawdown Duration      669 days 00:00:00
Avg. Drawdown Duration      149 days 00:00:00
# Trades                                  309
Win Rate [%]                        29.126214
Best Trade [%]                       3.362404
Worst Trade [%]                     -2.978859
Avg. Trade [%]                    